In [1]:
import urllib2  
from bs4 import BeautifulSoup  
import logging  
logging.basicConfig(level=logging.DEBUG)
import re
import pandas as pd
import requests


In [ ]:
def get_movie_value(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if field_name == 'Domestic:' or field_name == 'Foreign:' or field_name == 'Worldwide' or field_name == 'Weekend:':
        dtg_string = soup.find(text=re.compile(field_name))
        dtg = dtg_string.findNext().text
#         print dtg
        
        return dtg
    if next_sibling:
        return next_sibling.text 
    else:
        return None

In [ ]:
def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None
    
scrapePage(soup)

In [ ]:
import dateutil.parser

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date.year

def money_to_int(moneystring):
    if moneystring == None:
        return None
    elif moneystring.strip() == 'n/a':
        return None
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None
scrapePage(soup)

In [ ]:
# Let's get these again and format them all in one swoop

def scrapePage(soup):
    title_string = soup.find('title').text
    title = title_string.split('(')[0].strip()

    raw_release_date = get_movie_value(soup,'Release Date')
    release_date = to_date(raw_release_date)

    raw_domestic_total_gross = get_movie_value(soup,'Domestic:')
    domestic_total_gross = money_to_int(raw_domestic_total_gross)

    raw_foreign_total_gross = get_movie_value(soup,'Foreign:')
    foreign_total_gross = money_to_int(raw_foreign_total_gross)

    raw_worldwide_total_gross = get_movie_value(soup,'Worldwide')
    worldwide_total_gross = money_to_int(raw_worldwide_total_gross)

    raw_weekend_total_gross = get_movie_value(soup,'Weekend:')
    weekend_total_gross = money_to_int(raw_weekend_total_gross)

    raw_runtime = get_movie_value(soup,'Runtime')
    runtime = runtime_to_minutes(raw_runtime)
    
    distributor = get_movie_value(soup, 'Distributor:')
    
    rating = get_movie_value(soup,'MPAA Rating')

    headers = ['movie title', 'domestic total gross', 'foreign total gross', 
               'worldwide total gross', 'opening weekend gross',
               'release year', 'runtime (mins)', 'distributor', 'rating']

    movie_data = []
    movie_dict = dict(zip(headers, [title,
                                    domestic_total_gross,
                                    foreign_total_gross,
                                    worldwide_total_gross,
                                    weekend_total_gross,
                                    release_date,
                                    runtime,
                                    distributor,
                                    rating]))
    return movie_dict

    pprint(movie_data)

In [ ]:
#run this to scrape a page, need to add functionaltiy to get all of 2005

#Trying for 2005 only, this first part will find where to start


url2 = 'http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2005&p=.htm'
soup2 = BeautifulSoup(requests.get(url2).text,'html.parser')
all_links = soup2.findAll('a',href = re.compile('/yearly/chart/'))
listed = []
movieList = []
for link in all_links:
#     listed.append((link['href']))
#     print((link['href']))
    thelink = link['href']
#     print thelink, "TRYING"
    if 'page' in thelink and thelink not in listed and 'page=1' not in thelink:
#         print "here" 
        listed.append((link['href']))
        
for i in xrange(len(listed)+1):
    length = i+1
    url3 = 'http://www.boxofficemojo.com/yearly/chart/?page='+str(length)+'&view=releasedate&view2=domestic&yr=2005&p=.htm'
    print i+1, url3
    soup3 = BeautifulSoup(requests.get(url3).text,'html.parser')
    all_links = soup3.findAll('a',href = re.compile('/movies/\?*id='))
    count = 0
    for link in all_links:
        sleep = time.sleep(np.random.random())
        if count < 1:
            count += 1
        elif count > 2:
            break
        else:
            print((link['href']))
            movies = (link['href'])
            url = "http://www.boxofficemojo.com" + movies
            response = requests.get(url)
            page = response.text
            soup = BeautifulSoup(page, "lxml")
            movieInfo = scrapePage(soup)
            print movieInfo, "Movie Info"
            movieList.append(movieInfo)
            count += 1
    
# listed[:10]

In [ ]:
movieList = []
for year in xrange(2000,2006):
    url2 = 'http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr='+str(year)+'&p=.htm'
    soup2 = BeautifulSoup(requests.get(url2).text,'html.parser')
    all_links = soup2.findAll('a',href = re.compile('/yearly/chart/'))
    listed = []
    for link in all_links:
    #     listed.append((link['href']))
    #     print((link['href']))
        thelink = link['href']
    #     print thelink, "TRYING"
        if 'page' in thelink and thelink not in listed and 'page=1' not in thelink:
    #         print "here" 
            listed.append((link['href']))

    for i in xrange(len(listed)+1):
        length = i+1
        url3 = 'http://www.boxofficemojo.com/yearly/chart/?page='+str(length)+'&view=releasedate&view2=domestic&yr='+str(year)+'&p=.htm'
        print i+1, url3
        soup3 = BeautifulSoup(requests.get(url3).text,'html.parser')
        all_links = soup3.findAll('a',href = re.compile('/movies/\?*id='))
        count = 0
        for link in all_links:
            sleep = time.sleep(np.random.random())
            if count < 1:
                count += 1
#             elif count > 2:
#                 break
            else:
                print((link['href']))
                movies = (link['href'])
                url = "http://www.boxofficemojo.com" + movies
                response = requests.get(url)
                page = response.text
                soup = BeautifulSoup(page, "lxml")
                movieInfo = scrapePage(soup)
#                 print movieInfo, "Movie Info"
                movieList.append(movieInfo)
                count += 1


In [ ]:

movieList = []
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5)'}
for year in xrange(1980,2006):
    currentwd = "moviedata/"+str(year)
    if not os.path.exists(os.path.dirname(currentwd)):
        os.makedirs(os.path.dirname(currentwd))
        
    url2 = 'http://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr='+str(year)+'&p=.htm'
    resp = requests.get(url2, headers=headers)
    soup2 = BeautifulSoup(requests.get(url2).text,'html.parser')
    all_links = soup2.findAll('a',href = re.compile('/yearly/chart/'))
    listed = []
    for link in all_links:
        thelink = link['href']
        if 'page' in thelink and thelink not in listed and 'page=1' not in thelink:
            listed.append((link['href']))
    for i in xrange(len(listed)+1):
        length = i+1
        url3 = 'http://www.boxofficemojo.com/yearly/chart/?page='+str(length)+'&view=releasedate&view2=domestic&yr='+str(year)+'&p=.htm'
        print i+1, url3
        soup3 = BeautifulSoup(requests.get(url3).text,'html.parser')
        all_links = soup3.findAll('a',href = re.compile('/movies/\?*id='))
        count = 0
        for link in all_links:
            sleep = time.sleep(np.random.random())
            if count < 1:
                count += 1
            else: 
                movies = (link['href'])
                cleaned = movies[12:]
                url = "http://www.boxofficemojo.com" + movies
                print cleaned
                resp = requests.get(url, headers=headers)
                with open(os.path.join(currentwd, cleaned), 'wb') as f: # you may need to say 'wb' for Python2
                    f.write(resp.text.encode("utf-8"))
                count += 1


In [ ]:
#Go Through Folders years 1980 to 2005, 
#add each of the files to the list in the dictionary,
#then make it into a DataFrame, will need to join this with other datafile
#export to CSV
allFolder = os.listdir('moviedata')
allFolder
movieList = []
for year in allFolder:
    print year

    nineteenlist =os.listdir('moviedata/'+year)
#         print nineteenlist
    for html in nineteenlist:
#             print html
        if not os.path.isfile('moviedata/'+year+"/"+html):
            print "in if not"
            continue
        else:
            with open('moviedata/'+year+"/"+html) as f:
                print html, "html"
                html = f.read()
    #             page = response.text
                soup = BeautifulSoup(html, "lxml")
                movieInfo = scrapePage(soup)
                movieList.append(movieInfo)